In [1]:
%pip install pandas
%pip install xgboost
%pip install sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.
    ERROR: Command errored out with exit status 1:
     command: 'c:\Program Files\Python310\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\G994623\\AppData\\Local\\Temp\\pip-install-5lbgi91e\\sklearn_627e0fdc0e974b12abb2ba8cdadd0f3e\\setup.py'"'"'; __file__='"'"'C:\\Users\\G994623\\AppData\\Local\\Temp\\pip-install-5lbgi91e\\sklearn_627e0fdc0e974b12abb2ba8cdadd0f3e\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\G994623\AppData\Local\Temp\pip-pip-egg-info-2xusyzww'
         cwd: C:\Users\G994623\AppData\Local\Temp\pip-install-5lbgi91e\sklearn_627e0fdc0e974b12abb2ba8cdadd0f3e\
   

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post10.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post9.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [52]:
customers_data = pd.read_csv('data_sets/customers.csv')

# CHECK FOR NULL VALUES
customers_data.isnull().sum()

customers_data

,CUSTOMER_ID,x_customer_id,y_customer_id
0,7894622031164826,7.600289,92.486016
1,65009550197454,97.647808,36.539666
2,5381378528001886,98.643812,46.537793
3,344044685095332,2.814298,57.194127
4,5281794686066290,49.174496,88.879290
...,...,...,...
49995,2018185637039049,10.076680,69.035001
49996,7838600820947425,92.012646,86.350366
49997,1310171896045522,92.243870,50.776952
49998,3346506901987756,73.865160,43.451419


In [35]:
merchants_data = pd.read_csv('data_sets/merchants.csv')

# CHECK FOR NULL VALUES
merchants_data.isnull().sum()

MERCHANT_ID                           0
BUSINESS_TYPE                         0
MCC_CODE                              0
LEGAL_NAME                            0
FOUNDATION_DATE                       0
TAX_EXCEMPT_INDICATOR                 0
OUTLET_TYPE                           0
ACTIVE_FROM                           0
TRADING_FROM                          0
ANNUAL_TURNOVER_CARD                  0
ANNUAL_TURNOVER                       0
AVERAGE_TICKET_SALE_AMOUNT            0
PAYMENT_PERCENTAGE_FACE_TO_FACE       0
PAYMENT_PERCENTAGE_ECOM               0
PAYMENT_PERCENTAGE_MOTO               0
DEPOSIT_REQUIRED_PERCENTAGE           0
DEPOSIT_PERCENTAGE                    0
DELIVERY_SAME_DAYS_PERCENTAGE         0
DELIVERY_WEEK_ONE_PERCENTAGE          0
DELIVERY_WEEK_TWO_PERCENTAGE          0
DELIVERY_OVER_TWO_WEEKS_PERCENTAGE    0
dtype: int64

In [5]:
# DROP COLUMNS THAT ARE NOT NEEDED
merchants_data = merchants_data.drop(columns=['BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
                                                'TAX_EXCEMPT_INDICATOR', 'ACTIVE_FROM', 'TRADING_FROM', 'ANNUAL_TURNOVER_CARD',
                                                  'AVERAGE_TICKET_SALE_AMOUNT', 'DELIVERY_SAME_DAYS_PERCENTAGE',
                                                    'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE', 
                                                    'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE',
                                                    'PAYMENT_PERCENTAGE_FACE_TO_FACE','PAYMENT_PERCENTAGE_ECOM',
                                                    'PAYMENT_PERCENTAGE_MOTO'])

# CALCULATE DIFFERENCE IN DEPOSIT_REQUIRED_PERCENTAGE AND DEPOSIT_PERCENTAGE AND SAVE TO DEPOSIT_DIFF
merchants_data['DEPOSIT_DIFF'] = merchants_data['DEPOSIT_REQUIRED_PERCENTAGE'] - merchants_data['DEPOSIT_PERCENTAGE']

# DROP THE INDIVIDUAL DEPOSIT_PERCENTAGE AND DEPOSIT_REQUIRED_PERCENTAGE COLUMNS
merchants_data = merchants_data.drop(columns=['DEPOSIT_REQUIRED_PERCENTAGE', 'DEPOSIT_PERCENTAGE'])

merchants_data

,MERCHANT_ID,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
0,1b7557bb-208b-4fdc-ba91-eca98298541a,Ecommerce,351840227,0
1,6c948718-ea18-4d07-85d7-0916eb55e699,Ecommerce,411017547,0
2,f0151a49-7ac0-4258-93db-46bee2db6f66,Ecommerce,306887834,0
3,07089d03-0d63-4d24-a3f9-e651fda07cdc,Ecommerce,686209305,0
4,2c16629a-da48-4e69-9862-8fd521a53bbc,Ecommerce,384701644,0
...,...,...,...,...
30447,f58e8478-1566-4d1c-b0cb-b2106c8da3be,Face to Face and Ecommerce,304951,0
30448,68618d14-1e80-449d-95e3-6fa9e61901be,Face to Face and Ecommerce,735472,0
30449,e3cfd7f2-e2d1-40ea-b7b3-2583c57ecb3a,Face to Face and Ecommerce,645184,0
30450,3f8dff6f-d5b0-4963-99fa-3c64830fed23,Face to Face and Ecommerce,912396,0


In [53]:
terminals_data = pd.read_csv('data_sets/terminals.csv')

# CHECK FOR NULL VALUES
terminals_data.isnull().sum()

terminals_data

,TERMINAL_ID,x_terminal_id,y_terminal__id
0,9754224,41.536730,68.889911
1,37467861,61.864075,20.882662
2,20678215,70.530677,36.567633
3,12724021,92.961874,66.721471
4,2989793,38.756717,68.669003
...,...,...,...
4995,47999983,41.827704,36.312149
4996,13129180,11.208119,27.528779
4997,74645821,53.196734,82.410478
4998,65145440,59.074483,60.234741


In [50]:
transactions_test_data = pd.read_csv('data_sets/transactions_test.csv')

transactions_test_data = pd.merge(transactions_test_data, customers_data, on='CUSTOMER_ID')
transactions_test_data = pd.merge(transactions_test_data, terminals_data, on='TERMINAL_ID')

# # EUCLIDIAN DISTANCE
transactions_test_data['distance'] = (((transactions_test_data['x_customer_id'] - transactions_test_data['y_customer_id'])**2)
                                + ((transactions_test_data['x_terminal_id'] - transactions_test_data['y_terminal__id'])**2))**0.5
s
# CHECK FOR NULL VALUES
transactions_test_data.isnull().sum()

TX_ID                                        0
TX_TS                                        0
CUSTOMER_ID                                  0
TERMINAL_ID                                  0
TX_AMOUNT                                    0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT        0
TRANSACTION_CASHBACK_AMOUNT                  0
CARD_EXPIRY_DATE                             0
CARD_DATA                                    0
CARD_BRAND                                   0
TRANSACTION_TYPE                             0
TRANSACTION_STATUS                           0
FAILURE_CODE                             70009
FAILURE_REASON                           70009
TRANSACTION_CURRENCY                         0
CARD_COUNTRY_CODE                            0
MERCHANT_ID                                  1
IS_RECURRING_TRANSACTION                     0
ACQUIRER_ID                                  0
CARDHOLDER_AUTH_METHOD                       0
ID_JOIN                                      0
x_customer_id

In [51]:
# DROP COLUMNS
transactions_test_data = transactions_test_data.drop(columns=['TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA', 'CARD_EXPIRY_DATE',
                                                                  'TRANSACTION_STATUS', 'TRANSACTION_TYPE',
                                                                  'CARD_COUNTRY_CODE','TRANSACTION_CURRENCY',
                                                                  'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
                                                                  'x_customer_id', 'y_customer_id',
                                                                  'x_terminal_id', 'y_terminal__id'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_test_data = transactions_test_data.loc[:, (transactions_test_data != 0.0).any(axis=0)]

transactions_test_data

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,distance
0,09324d812ba7915c3f791e973db293ad50db70d8,4635360198955981,4641801,0.0,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0,53.737115
1,304acb4a3f01e587cde1afaec3ca31d0cdc97f2d,2759476206244285,4641801,0.0,39ee690d-8575-4b1c-b7df-2b4415dcf914,True,ACQ3,Online PIN,7570.0,56.109865
2,88404703377a385f41a0f66c6c7e517392c18389,2759476206244285,4641801,0.0,1e8baf90-a292-44e0-87bf-fd232c530a0f,False,ACQ1,No CVM performed,11677.0,56.109865
3,1f875dc9fa75d9aab78f7c9f5880ada726ed9dff,511269827752665,4641801,0.0,22a90b6f-9768-4df7-be53-d83d86347a35,False,ACQ3,Online PIN,7626.0,54.175138
4,660a47e7b1422ffbcd8581da9aae541db84d7a49,2271650133604365,4641801,0.0,0bd7ce27-cef4-42bd-944b-337f79c0e38f,False,ACQ1,Online PIN,6589.0,52.398684
...,...,...,...,...,...,...,...,...,...,...
71134,5bc9c2dc33f5e62799c922a33ccb43d438492411,6791500339924992,10515200,0.0,32fe97cc-a518-4379-816b-e571778715ac,False,ACQ3,Offline plaintext PIN,8188.0,128.938386
71135,a769750eb2c8dbfbdb69204906146d64821f3c46,3546514278641005,10515200,0.0,c727b224-2dec-44a2-a766-e357e5dfc19a,False,ACQ2,Offline enciphered PIN and signature,6983.0,123.017462
71136,bbe806307501e6bbcac927fd4846988163fed75d,7748050858955326,10515200,0.0,9057c5d2-7f9f-407f-aea4-3b2f5d1a4da1,False,ACQ3,Online PIN,7825.0,124.723113
71137,e0ba811f6857529659d191d020c8e5888c3f4bb3,8615298785123346,53516402,0.0,d130ca92-4067-4c33-86af-60ef33aa3bde,False,ACQ3,Online PIN,16817.0,90.382882


In [10]:
# CHECK FOR NULL VALUES AGAIN
transactions_test_data.isnull().sum()

TX_ID                          0
CUSTOMER_ID                    0
TERMINAL_ID                    0
TRANSACTION_CASHBACK_AMOUNT    0
MERCHANT_ID                    1
IS_RECURRING_TRANSACTION       0
ACQUIRER_ID                    0
CARDHOLDER_AUTH_METHOD         0
ID_JOIN                        0
dtype: int64

In [55]:
transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')

transactions_train_data = pd.merge(transactions_train_data, customers_data, on='CUSTOMER_ID')
transactions_train_data = pd.merge(transactions_train_data, terminals_data, on='TERMINAL_ID')

# EUCLIDIAN DISTANCE
transactions_train_data['distance'] = (((transactions_train_data['x_customer_id'] - transactions_train_data['y_customer_id'])**2)
                                + ((transactions_train_data['x_terminal_id'] - transactions_train_data['y_terminal__id'])**2))**0.5


# CHECK FOR NULL VALUES
transactions_train_data.isnull().sum()

C:\Users\G994623\AppData\Local\Temp\ipykernel_31112\2402122785.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')


TX_ID                                          0
TX_TS                                          0
CUSTOMER_ID                                    0
TERMINAL_ID                                    0
TX_AMOUNT                                      0
TX_FRAUD                                       0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT          0
TRANSACTION_CASHBACK_AMOUNT                    0
CARD_EXPIRY_DATE                               0
CARD_DATA                                      0
CARD_BRAND                                     0
TRANSACTION_TYPE                               0
TRANSACTION_STATUS                             0
FAILURE_CODE                             1049208
FAILURE_REASON                           1049208
TRANSACTION_CURRENCY                           0
CARD_COUNTRY_CODE                              0
MERCHANT_ID                                    0
IS_RECURRING_TRANSACTION                       0
ACQUIRER_ID                                    1
CARDHOLDER_AUTH_METH

In [56]:
# Clean data:
transactions_train_data = transactions_train_data.drop(columns=['TX_ID', 'TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                 'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA',
                                                                   'CARD_EXPIRY_DATE', 'TRANSACTION_STATUS', 'TRANSACTION_TYPE',
                                                                   'CARD_COUNTRY_CODE','TRANSACTION_CURRENCY',
                                                                  'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
                                                                  'x_customer_id', 'y_customer_id',
                                                                  'x_terminal_id', 'y_terminal__id'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_train_data = transactions_train_data.loc[:, (transactions_train_data != 0.0).any(axis=0)]

# DROP ROW IF A VALUE IS NAN
transactions_train_data = transactions_train_data.dropna()

transactions_train_data

,CUSTOMER_ID,TERMINAL_ID,TX_FRAUD,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,distance
0,7323088226725338,55875360,0,0.00,136def43-2ef3-4bc9-a8c2-60598c316e9c,False,ACQ1,Online PIN,11.709690
1,7323088226725338,55875360,0,0.00,d2956448-407a-4cec-8259-7dbd797f1fd7,False,ACQ3,Offline plaintext PIN,11.709690
2,4362790219307564,55875360,0,0.00,c072e147-18dc-456a-8132-124a9ebe313b,False,ACQ2,Offline enciphered PIN,11.634230
3,2376754785900600,55875360,0,0.00,ae6399bd-6f2c-4ecf-b376-9d82a90d4ab3,False,ACQ2,Offline enciphered PIN,11.229510
4,2376754785900600,55875360,1,5.63,7eb03e7c-220b-4fa5-9164-b69ab72aa87e,False,ACQ3,Online PIN,11.229510
...,...,...,...,...,...,...,...,...,...
1066435,7760361189551290,26508045,0,0.00,b93498b2-e7c3-4170-8a92-973110ce6d1a,False,ACQ1,Offline plaintext PIN,19.404982
1066436,4712914414618850,26508045,0,0.00,2f5b7a3b-bf6d-4714-a073-341128e3ea67,False,ACQ2,Online PIN,20.211619
1066437,3473580532934853,26508045,0,0.00,8c75bf43-03e2-4f39-8b36-2c6688604e94,False,ACQ1,No CVM performed,22.869618
1066438,84525978769760,26508045,0,0.00,e9ab9513-81cf-4da9-9cc1-93330b92aa10,False,ACQ1,Offline enciphered PIN,26.423300


In [13]:
# CHECK FOR NULL VALUES AGAIN
transactions_train_data.isnull().sum()

CUSTOMER_ID                    0
TERMINAL_ID                    0
TX_FRAUD                       0
TRANSACTION_CASHBACK_AMOUNT    0
MERCHANT_ID                    0
IS_RECURRING_TRANSACTION       0
ACQUIRER_ID                    0
CARDHOLDER_AUTH_METHOD         0
dtype: int64

In [57]:
# MERGE TABLES FOR TRAINING DATA
train_data = pd.merge(transactions_train_data, merchants_data, on='MERCHANT_ID', how='left')

train_data

,CUSTOMER_ID,TERMINAL_ID,TX_FRAUD,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,distance,BUSINESS_TYPE,...,AVERAGE_TICKET_SALE_AMOUNT,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE,DELIVERY_SAME_DAYS_PERCENTAGE,DELIVERY_WEEK_ONE_PERCENTAGE,DELIVERY_WEEK_TWO_PERCENTAGE,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE
0,7323088226725338,55875360,0,0.00,136def43-2ef3-4bc9-a8c2-60598c316e9c,False,ACQ1,Online PIN,11.709690,Sole Proprietorships,...,59,37,63,0,69,69,24,20,20,36
1,7323088226725338,55875360,0,0.00,d2956448-407a-4cec-8259-7dbd797f1fd7,False,ACQ3,Offline plaintext PIN,11.709690,Sole Proprietorships,...,119,72,28,0,23,23,21,27,22,30
2,4362790219307564,55875360,0,0.00,c072e147-18dc-456a-8132-124a9ebe313b,False,ACQ2,Offline enciphered PIN,11.634230,Corporations,...,571,42,58,0,59,59,21,26,23,30
3,2376754785900600,55875360,0,0.00,ae6399bd-6f2c-4ecf-b376-9d82a90d4ab3,False,ACQ2,Offline enciphered PIN,11.229510,Corporations,...,545,70,20,10,80,20,29,23,21,27
4,2376754785900600,55875360,1,5.63,7eb03e7c-220b-4fa5-9164-b69ab72aa87e,False,ACQ3,Online PIN,11.229510,Sole Proprietorships,...,16,81,19,0,24,24,26,27,29,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066434,7760361189551290,26508045,0,0.00,b93498b2-e7c3-4170-8a92-973110ce6d1a,False,ACQ1,Offline plaintext PIN,19.404982,Limited Liability Company (LLC),...,157,57,43,0,69,69,27,20,21,32
1066435,4712914414618850,26508045,0,0.00,2f5b7a3b-bf6d-4714-a073-341128e3ea67,False,ACQ2,Online PIN,20.211619,Limited Liability Company (LLC),...,155,100,0,0,17,17,20,24,20,36
1066436,3473580532934853,26508045,0,0.00,8c75bf43-03e2-4f39-8b36-2c6688604e94,False,ACQ1,No CVM performed,22.869618,Limited Liability Company (LLC),...,100,44,56,0,2,2,26,27,27,20
1066437,84525978769760,26508045,0,0.00,e9ab9513-81cf-4da9-9cc1-93330b92aa10,False,ACQ1,Offline enciphered PIN,26.423300,Limited Liability Company (LLC),...,25,78,22,0,0,0,25,25,23,27


In [15]:
# PRINT ANY ENTRIES THAT ARE NA
print(train_data[train_data.isna().any(axis=1)])

Empty DataFrame
Columns: [CUSTOMER_ID, TERMINAL_ID, TX_FRAUD, TRANSACTION_CASHBACK_AMOUNT, MERCHANT_ID, IS_RECURRING_TRANSACTION, ACQUIRER_ID, CARDHOLDER_AUTH_METHOD, OUTLET_TYPE, ANNUAL_TURNOVER, DEPOSIT_DIFF]
Index: []


In [58]:
# MERGE TABLES FOR TESTING DATA
test_data = pd.merge(transactions_test_data, merchants_data, on='MERCHANT_ID', how='left')

test_data

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,distance,...,AVERAGE_TICKET_SALE_AMOUNT,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE,DELIVERY_SAME_DAYS_PERCENTAGE,DELIVERY_WEEK_ONE_PERCENTAGE,DELIVERY_WEEK_TWO_PERCENTAGE,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE
0,09324d812ba7915c3f791e973db293ad50db70d8,4635360198955981,4641801,0.0,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0,53.737115,...,82.0,39.0,61.0,0.0,17.0,17.0,22.0,20.0,21.0,37.0
1,304acb4a3f01e587cde1afaec3ca31d0cdc97f2d,2759476206244285,4641801,0.0,39ee690d-8575-4b1c-b7df-2b4415dcf914,True,ACQ3,Online PIN,7570.0,56.109865,...,11.0,56.0,44.0,0.0,23.0,23.0,25.0,22.0,25.0,28.0
2,88404703377a385f41a0f66c6c7e517392c18389,2759476206244285,4641801,0.0,1e8baf90-a292-44e0-87bf-fd232c530a0f,False,ACQ1,No CVM performed,11677.0,56.109865,...,133.0,100.0,0.0,0.0,36.0,36.0,26.0,25.0,23.0,26.0
3,1f875dc9fa75d9aab78f7c9f5880ada726ed9dff,511269827752665,4641801,0.0,22a90b6f-9768-4df7-be53-d83d86347a35,False,ACQ3,Online PIN,7626.0,54.175138,...,447.0,65.0,35.0,0.0,23.0,23.0,25.0,27.0,28.0,20.0
4,660a47e7b1422ffbcd8581da9aae541db84d7a49,2271650133604365,4641801,0.0,0bd7ce27-cef4-42bd-944b-337f79c0e38f,False,ACQ1,Online PIN,6589.0,52.398684,...,7.0,47.0,53.0,0.0,20.0,20.0,29.0,23.0,21.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71134,5bc9c2dc33f5e62799c922a33ccb43d438492411,6791500339924992,10515200,0.0,32fe97cc-a518-4379-816b-e571778715ac,False,ACQ3,Offline plaintext PIN,8188.0,128.938386,...,11.0,61.0,39.0,0.0,25.0,25.0,29.0,20.0,21.0,30.0
71135,a769750eb2c8dbfbdb69204906146d64821f3c46,3546514278641005,10515200,0.0,c727b224-2dec-44a2-a766-e357e5dfc19a,False,ACQ2,Offline enciphered PIN and signature,6983.0,123.017462,...,103.0,65.0,24.0,11.0,21.0,21.0,22.0,21.0,26.0,31.0
71136,bbe806307501e6bbcac927fd4846988163fed75d,7748050858955326,10515200,0.0,9057c5d2-7f9f-407f-aea4-3b2f5d1a4da1,False,ACQ3,Online PIN,7825.0,124.723113,...,49.0,44.0,56.0,0.0,24.0,89.0,25.0,20.0,29.0,26.0
71137,e0ba811f6857529659d191d020c8e5888c3f4bb3,8615298785123346,53516402,0.0,d130ca92-4067-4c33-86af-60ef33aa3bde,False,ACQ3,Online PIN,16817.0,90.382882,...,88.0,51.0,49.0,0.0,54.0,54.0,24.0,22.0,26.0,28.0


In [17]:
# PRINT ANY ENTRIES THAT ARE NA
test_data[test_data.isna().any(axis=1)]

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
8201,ddaa070acea087eae360225e92c1609cea905e43,1513339476778402,77821795,0.0,NaN,False,ACQ3,Online PIN,0.0,NaN,NaN,NaN


In [18]:
# Separate target prediction
y = train_data.TX_FRAUD
X = train_data.drop(['TX_FRAUD' ,'CUSTOMER_ID',  'TERMINAL_ID','MERCHANT_ID'], axis=1)
test_data_TX_ID = test_data.TX_ID
X_test = test_data.drop(['CUSTOMER_ID',  'TERMINAL_ID','MERCHANT_ID', 'ID_JOIN', 'TX_ID'], axis=1)
test_data_TX_ID.head()

0    09324d812ba7915c3f791e973db293ad50db70d8
1    a17bc798bacadbde6cd7928688e2453fa6023d67
2    dee3efed3600425f60eee0e9d363f9bac226e12a
3    e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7
4    b25981de8b5dcdc1c38de111a21312115c34199d
Name: TX_ID, dtype: object

In [19]:
# Divide data into training data and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
object_cols_test_data = [col for col in X_train.columns if X_train[col].dtype == "object"]

In [20]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols].astype(str)))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols].astype(str)))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols].astype(str)))

C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_test.index = X_test.index

In [22]:
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols_test_data, axis=1)

In [23]:
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

In [24]:
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

In [25]:
OH_X_train.head()

,TRANSACTION_CASHBACK_AMOUNT,ANNUAL_TURNOVER,DEPOSIT_DIFF,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16,17
586284,0.0,594342838,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1003473,0.0,245774,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
527600,0.0,16357,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
258650,0.0,430950,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
608642,0.0,341877,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [26]:
reg_model = XGBRegressor(eval_metric=mean_absolute_error, learning_rate=0.05, n_estimators=500)
reg_model.fit(OH_X_train, y_train, eval_set=[(OH_X_valid, y_valid)], early_stopping_rounds=5, verbose=False)

C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDt

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x0000019FC6C40C10>,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [27]:
reg_model.evals_result()

{'validation_0': OrderedDict([('rmse',
               [0.1581709986230937,
                0.15815415122440113,
                0.15813904254564945,
                0.15812575770346385,
                0.15811403879371924,
                0.1581026012331725,
                0.15809343846851567,
                0.15808496764650642,
                0.15807804098986364,
                0.15807173394425877,
                0.1580650172478926,
                0.15805981364645785,
                0.15805471852388883,
                0.1580510913706082,
                0.158048440095743,
                0.15804465037753637,
                0.15804128460735659,
                0.15803868248493366,
                0.15803604424497192,
                0.15803490646423693,
                0.15803276766785967,
                0.15803186294503044,
                0.15803198131512192,
                0.1580304672738566,
                0.15802969407238557,
                0.15802905329596306,
      

In [28]:
answers = reg_model.predict(OH_X_test)
answers

C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\G994623\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDt

array([0.02383485, 0.02248139, 0.02205592, ..., 0.02243544, 0.02337503,
       0.02267178], dtype=float32)

In [29]:
output = pd.DataFrame({"TX_ID": test_data_TX_ID, "TX_FRAUD": answers})
output.set_index("TX_ID", inplace=True)
output

,TX_FRAUD
TX_ID,
09324d812ba7915c3f791e973db293ad50db70d8,0.023835
a17bc798bacadbde6cd7928688e2453fa6023d67,0.022481
dee3efed3600425f60eee0e9d363f9bac226e12a,0.022056
e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.022899
b25981de8b5dcdc1c38de111a21312115c34199d,0.023162
...,...
b8a998afbc7152514b1ae19d3e74c1566fb9547b,0.022249
dd1f46bf3fe4cbe806c000d33c9491595aad9706,0.023000
ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,0.022435


In [30]:
max_a = output.TX_FRAUD.max()
min_a = output.TX_FRAUD.min()
min_norm = 0
max_norm =1
output['TX_FRAUD'] = round((output.TX_FRAUD- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm , 3)
output

,TX_FRAUD
TX_ID,
09324d812ba7915c3f791e973db293ad50db70d8,0.023
a17bc798bacadbde6cd7928688e2453fa6023d67,0.021
dee3efed3600425f60eee0e9d363f9bac226e12a,0.020
e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.022
b25981de8b5dcdc1c38de111a21312115c34199d,0.022
...,...
b8a998afbc7152514b1ae19d3e74c1566fb9547b,0.021
dd1f46bf3fe4cbe806c000d33c9491595aad9706,0.022
ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,0.021


In [31]:
output.to_csv('submission.csv')